In [9]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
sys.path.append('/Users/ratzenboe/Documents/work/code/StarPipeline/')
sys.path.append('/Users/ratzenboe/Documents/work/code/StarPipeline/dev')
from dev.isochrone_nn_interpolator import train_with_validation, FastNNInterpolator

In [ ]:
parsec_file_path = '/Users/ratzenboe/Documents/work/data_local/parsec_evo_tracks/all_tracks_Pv1/parsec_tracks_postprocessed.csv'

In [3]:
df_parsec = pd.read_csv(parsec_file_path)

In [6]:
X = df_parsec[['logAge', 'mass', 'Z']].values
y = df_parsec[['logL', 'logT', 'logR']].values

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=0
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=0
)

In [12]:
model, input_scaler, output_scaler = train_with_validation(
    X_train, y_train, X_val, y_val
)

Epoch 0: Train Loss 1.000489, Val Loss 0.976252
Epoch 100: Train Loss 0.423742, Val Loss 0.385908
Epoch 200: Train Loss 0.363224, Val Loss 0.325777
Epoch 300: Train Loss 0.290455, Val Loss 0.244767
Epoch 400: Train Loss 0.241029, Val Loss 0.194955
Epoch 500: Train Loss 0.210311, Val Loss 0.169051
Epoch 600: Train Loss 0.188223, Val Loss 0.148895
Epoch 700: Train Loss 0.171486, Val Loss 0.132388
Epoch 800: Train Loss 0.157703, Val Loss 0.122721
Epoch 900: Train Loss 0.147044, Val Loss 0.112572


In [13]:
fast_interp = FastNNInterpolator(model, input_scaler, output_scaler)